In [1]:
import os
import glob
import h5py
import re
import json
import traceback 
import copy
import itertools
import time

import pylab as pl
import numpy as np
import pandas as pd
import seaborn as sns
import caiman as cm

from scipy.stats import stats
import matplotlib.patches as patches   

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


def label_figure(fig, fig_id):
    fig.text(0, 1, fig_id, ha='left', va='top', fontsize=10)
    
from caiman.source_extraction.cnmf.cnmf import load_CNMF

/home/julianarhee/anaconda2/envs/caiman/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.incremental_pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/julianarhee/anaconda2/envs/caiman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/julianarhee/anaconda2/envs/caiman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy,

In [8]:
import utils as util

In [2]:
%matplotlib notebook

# Select data source

In [34]:
import imp
imp.reload(util)

<module 'utils' from '/home/julianarhee/Repositories/2p-pipeline/pipeline/python/traces/utils.py'>

In [38]:
rootdir = '/n/coxfs01/2p-data'
animalid = 'JC084' #'JC084'
session = '20190522' #'20190522' #'20190505_JC083'
session_dir = os.path.join(rootdir, animalid, session)

fov = 'FOV1_zoom2p0x'
experiment = 'gratings' #'retino_run2'


dataid = '|'.join([animalid, session, fov, experiment])
print("*** Dataset: %s ***" % dataid)

# -------------------------------------------
# Select extraction type (seeded or patches)
# ------------------------------------------
seeded = False
roiid = None
traceid = None
if seeded:
    assert traceid is not None or roiid is not None, "No roiid or traceid specified..."
    if roiid is None and traceid is not None:
        roiid = util.get_roiid_from_traceid(animalid, session, fov, run_type=experiment, traceid=traceid)
    extraction_id = 'seeded_%s_%s' % (roiid, experiment)
else:
    extraction_id = 'patches_%s' % (experiment)

print("Requested: %s" % extraction_id)


*** Dataset: JC084|20190522|FOV1_zoom2p0x|gratings ***
Requested: patches_gratings


In [40]:
# -------------------------------------------
# Load extracted traces
# ------------------------------------------
caiman_results_dir = os.path.join(session_dir, fov, 'caiman_results', experiment)
found_results_files = glob.glob(os.path.join(caiman_results_dir, '%s_results.hdf5' % extraction_id))
assert len(found_results_files)==1, "Error:  too many/few results files: %s" % str(found_results_files)
results_fpath = found_results_files[0]

print("...loading %s" % results_fpath)
cnm = load_CNMF(results_fpath, n_processes=2)
print("%i of %i components passed initial eval" % (len(cnm.estimates.idx_components), cnm.estimates.A.shape[-1]))
print(cnm.params.quality)


...loading /n/coxfs01/2p-data/JC084/20190522/FOV1_zoom2p0x/caiman_results/gratings/patches_gratings_results.hdf5


(262144, 1009)

In [107]:
outdir = os.path.join(caiman_results_dir, extraction_id)
if not os.path.exists(outdir):
    os.mkdirs(outdir)
print(outdir)

/n/coxfs01/2p-data/JC084/20190522/FOV1_zoom2p0x/caiman_results/gratings/patches_gratings


#### Visualize initial evaluation

In [104]:
def load_roi_masks(animalid, session, fov, rois=None, rootdir='/n/coxfs01/2p-data'):
    '''
    Loads MANUAL ROIs (e.g., .../ROIs/rois001/masks.hdf5)
    In Python, loaded masks are shape (nrois, d1, d2)
    '''
    masks=None; zimg=None;
    mask_fpath = glob.glob(os.path.join(rootdir, animalid, session, 'ROIs', '%s*' % rois, 'masks.hdf5'))[0]
    try:
        mfile = h5py.File(mask_fpath, 'r')
        fkey = list(mfile.keys())[0]
        masks = mfile[fkey]['masks']['Slice01'][:] #.T

        zimg = mfile[fkey]['zproj_img']['Slice01'][:] #.T
        zimg.shape
    except Exception as e:
        print("error loading masks")
    finally:
        mfile.close()
        
    return masks, zimg

In [65]:
# Load z-projection image for visualization
msk, zimg = load_roi_masks(animalid, session, fov, rois='rois001')

# Get good components
idx_components = cnm.estimates.idx_components
dims = cnm.dims



# Evaluate components again

In [112]:
# Save kept components w/ evaluation key 
evaluation_params = sorted(['min_SNR', 'min_cnn_thr', 'cnn_lowest', 'rval_thr', 'rval_lowest', 'gSig_range'])
eval_str = '_'.join(['%s_%s' % (k, str(v)) for k, v in cnm.params.quality.items() if k in evaluation_params])
print(eval_str)

# Plot
fig, ax = pl.subplots(figsize=(6,6))
cm.utils.visualization.plot_contours(cnm.estimates.A[:, idx_components].toarray(), zimg.T, 
                                     cmap='gray', colors='r', display_numbers=False)
figname = 'pass__%s' % eval_str
pl.savefig(os.path.join(outdir, '%s.png' % figname))
print(outdir, figname)

cnn_lowest_0.05_gSig_range_None_min_SNR_2.0_min_cnn_thr_0.99_rval_lowest_-1_rval_thr_0.85


In [116]:
n_processes = 1

#%% start a cluster for parallel processing 
# (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=n_processes, single_thread=False)
print("--- running on %i processes ---" % n_processes)
#dview=None 
print("--- dview: ", dview)


--- running on 1 processes ---
--- dview:  <multiprocessing.pool.Pool object at 0x7f5edc360a90>


In [117]:
#%% Get memmapping info (motion correction, mmap fil einfo, preprocessing info)
preprocess_fpath = results_fpath.replace('results.hdf5', 'processing_params.json')
with open(preprocess_fpath, 'r') as f:
    preprocess_info = json.load(f)
if '-' in os.path.split(preprocess_info['source'])[-1]:
    tmp_parent, tmp_fn = os.path.split(preprocess_info['source'])
    fix_fn = tmp_fn.replace('-', '_')
    mmap_fname = os.path.join(tmp_parent, fix_fn)
    preprocess_info['source'] = mmap_fname
    with open(preprocess_fpath, 'w') as f:
        json.dump(preprocess_info, f, indent=4)
mmap_fname = preprocess_info['source']
mmap_prefix = os.path.splitext(os.path.split(mmap_fname)[-1])[0].split('_d1_')[0]
#print(mmap_fname)
#print(mmap_prefix)
print(preprocess_info)


{'quantileMin': 10, 'frames_window_sec': 30.0, 'ds_factor': 1.0, 'fr': 44.65, 'frames_window': 1340, 'source': '/n/coxfs01/2p-data/JC084/20190522/FOV1_zoom2p0x/caiman_results/gratings/memmap/JC084_20190522_FOV1_zoom2p0x_gratings_Yr_d1_512_d2_512_d3_1_order_C_frames_142910_.mmap'}


In [118]:
# Load data
Yr, dims, T = cm.load_memmap(mmap_fname)
images = np.reshape(Yr.T, [T] + list(dims), order='F') #np.reshape(Yr, dims + (T,), order='F')
print("Loaded data:", images.shape)


Loaded data: (142910, 512, 512)


In [119]:

# Evaluate components 
print("Evaluating components...")
# parameters for component evaluation
min_SNR = 2.0               # signal to noise ratio for accepting a component
rval_thr = 0.85              # space correlation threshold for accepting a component
min_cnn_thr = 0.99          # threshold for CNN based classifier
cnn_lowest = 0.05           # neurons with cnn probability lower than this value are rejected
#cnm_seeded.estimates.restore_discarded_components()
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': rval_thr,
                           'use_cnn': True,
                           'min_cnn_thr': min_cnn_thr,
                           'cnn_lowest': cnn_lowest})

#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier
start_t = time.time()
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
end_t = time.time() - start_t
print("--> evaluation - Elapsed time: {0:.2f}sec".format(end_t))
print("Discarding %i of %i initially selected components." % (len(cnm.estimates.idx_components_bad), cnm.estimates.A.shape[-1]))
#cnm.estimates.select_components(use_object=True)


Evaluatnig components...
USING MODEL:/home/julianarhee/caiman_data/model/cnn_model.json
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


1009/1009 [==============================] - 1s 1ms/sample
--> evaluation - Elapsed time: 11870.12sec
Discarding 377 of 1009 initially selected components.


In [120]:
cnm.params.quality

{'SNR_lowest': 0.5,
 'cnn_lowest': 0.05,
 'gSig_range': None,
 'min_SNR': 2.0,
 'min_cnn_thr': 0.99,
 'rval_lowest': -1,
 'rval_thr': 0.85,
 'use_cnn': True}

In [121]:
print("%i of %i components passed initial eval" % (len(cnm.estimates.idx_components), cnm.estimates.A.shape[-1]))
print(cnm.params.quality)


632 of 1009 components passed initial eval
{'SNR_lowest': 0.5, 'cnn_lowest': 0.05, 'gSig_range': None, 'min_SNR': 2.0, 'min_cnn_thr': 0.99, 'rval_lowest': -1, 'rval_thr': 0.85, 'use_cnn': True}


In [122]:
# Save kept components w/ evaluation key 
evaluation_params = sorted(['min_SNR', 'min_cnn_thr', 'cnn_lowest', 'rval_thr', 'rval_lowest', 'gSig_range'])
eval_str = '_'.join(['%s_%s' % (k, str(v)) for k, v in cnm.params.quality.items() if k in evaluation_params])
print(eval_str)

# Plot
fig, axes = pl.subplots(1, 2, figsize=(6,6))
ax = axes[0]
cm.utils.visualization.plot_contours(cnm.estimates.A[:, idx_components].toarray(), zimg.T, 
                                     cmap='gray', colors='r', display_numbers=False)
ax = axes[1]
cm.utils.visualization.plot_contours(cnm.estimates.A[:, idx_components_bad].toarray(), zimg.T, 
                                     cmap='gray', colors='gray', display_numbers=False)

figname = 'pass__%s' % eval_str
pl.savefig(os.path.join(outdir, '%s.png' % figname))
print(outdir, figname)

cnn_lowest_0.05_gSig_range_None_min_SNR_2.0_min_cnn_thr_0.99_rval_lowest_-1_rval_thr_0.85


<IPython.core.display.Javascript object>

NameError: name 'idx_components_bad' is not defined